<a href="https://colab.research.google.com/github/huseyindas/shap-xgboost-financial-analysis/blob/master/shap_credit_card_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score, confusion_matrix
from xgboost import XGBClassifier

In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "UCI_Credit_Card.csv"
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "uciml/default-of-credit-card-clients-dataset",
  file_path
)

100%|██████████| 0.98M/0.98M [00:00<00:00, 1.93MB/s]

Extracting zip of UCI_Credit_Card.csv...


In [3]:
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000.0,1,3,1,39,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,29997,150000.0,1,3,2,43,-1,-1,-1,-1,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,29998,30000.0,1,2,2,37,4,3,2,-1,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,29999,80000.0,1,3,1,41,1,-1,0,0,...,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [5]:
df.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [6]:
print('Missing values in each column:')
df.isnull().sum()

Missing values in each column:


,0
ID,0
LIMIT_BAL,0
SEX,0
EDUCATION,0
MARRIAGE,0
AGE,0
PAY_0,0
PAY_2,0
PAY_3,0
PAY_4,0


In [7]:
# 1) Clean categories
df['EDUCATION'] = df['EDUCATION'].replace({0: 4, 5: 4, 6: 4})
df['MARRIAGE']  = df['MARRIAGE'].replace({0: 3})

# 2) One-hot (cleaner for papers)
df = pd.get_dummies(df, columns=['SEX','EDUCATION','MARRIAGE'], drop_first=True)

# 3) Split X/y (drop ID!)
y = df['default.payment.next.month'].astype(int)
X = df.drop(columns=['default.payment.next.month', 'ID'], errors='ignore')

In [8]:
df.head()

,ID,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month,SEX_2,EDUCATION_2,EDUCATION_3,EDUCATION_4,MARRIAGE_2,MARRIAGE_3
0,1,20000.0,24,2,2,-1,-1,-2,-2,3913.0,...,0.0,0.0,0.0,1,True,True,False,False,False,False
1,2,120000.0,26,-1,2,0,0,0,2,2682.0,...,1000.0,0.0,2000.0,1,True,True,False,False,True,False
2,3,90000.0,34,0,0,0,0,0,0,29239.0,...,1000.0,1000.0,5000.0,0,True,True,False,False,True,False
3,4,50000.0,37,0,0,0,0,0,0,46990.0,...,1100.0,1069.0,1000.0,0,True,True,False,False,False,False
4,5,50000.0,57,-1,0,-1,0,0,0,8617.0,...,9000.0,689.0,679.0,0,False,True,False,False,False,False


In [9]:
# 1) Split
y = df["default.payment.next.month"].astype(int)
X = df.drop(columns=["default.payment.next.month", "ID"], errors="ignore")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [10]:
# 2) Handle imbalance (scale_pos_weight)
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

In [11]:
# 3) Baseline model
model = XGBClassifier(
    n_estimators=600,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    min_child_weight=1,
    objective="binary:logistic",
    eval_metric="auc",
    scale_pos_weight=scale_pos_weight,
    n_jobs=-1,
    random_state=42
)

model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=600,
              n_jobs=-1, num_parallel_tree=None, ...)

In [12]:
# 4) Predictions
proba = model.predict_proba(X_test)[:, 1]
pred  = (proba >= 0.5).astype(int)

In [13]:
# 5) Metrics
roc_auc = roc_auc_score(y_test, proba)
pr_auc  = average_precision_score(y_test, proba)
f1      = f1_score(y_test, pred)
prec    = precision_score(y_test, pred)
rec     = recall_score(y_test, pred)
cm      = confusion_matrix(y_test, pred)

print("scale_pos_weight:", scale_pos_weight)
print("ROC-AUC:", roc_auc)
print("PR-AUC:", pr_auc)
print("F1:", f1, "Precision:", prec, "Recall:", rec)
print("Confusion matrix:\n", cm)

scale_pos_weight: 3.520625353173856
ROC-AUC: 0.7727146649344927
PR-AUC: 0.5464343321114901
F1: 0.529373368146214 Precision: 0.46689694876223375 Recall: 0.6111529766390355
Confusion matrix:
 [[3747  926]
 [ 516  811]]


In [14]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

thresholds = np.linspace(0.1, 0.9, 81)
rows = []
for t in thresholds:
    p = (proba >= t).astype(int)
    rows.append((t,
                 precision_score(y_test, p),
                 recall_score(y_test, p),
                 f1_score(y_test, p)))

best_f1 = max(rows, key=lambda x: x[3])
best_rec = max(rows, key=lambda x: x[2])

print("Best-F1 threshold:", best_f1)
print("Best-Recall threshold:", best_rec)

Best-F1 threshold: (np.float64(0.56), 0.5184926727145848, 0.5599095704596835, 0.5384057971014493)
Best-Recall threshold: (np.float64(0.1), 0.23259943181818182, 0.987189148455162, 0.3764908751257365)


In [15]:
import shap
import matplotlib.pyplot as plt
import numpy as np

# Train verisi üzerinden explainer (TreeExplainer en doğru ve hızlı)
explainer = shap.TreeExplainer(model)

# SHAP values (test setten bir örneklem yeter)
X_shap = X_test.sample(n=2000, random_state=42) if len(X_test) > 2000 else X_test
shap_values = explainer.shap_values(X_shap)

# 1) Global: bar plot (Top-10)
plt.figure()
shap.summary_plot(shap_values, X_shap, plot_type="bar", max_display=10, show=False)
plt.tight_layout()
plt.savefig("shap_global_top10.png", dpi=300, bbox_inches="tight")
plt.close()

# 2) Local: waterfall (tek örnek)
i = 0  # istersen risk skoru yüksek birini seçmek için aşağıya bak
x_row = X_test.iloc[i:i+1]
sv_row = explainer(x_row)

plt.figure()
shap.plots.waterfall(sv_row[0], max_display=12, show=False)
plt.tight_layout()
plt.savefig("shap_local_waterfall.png", dpi=300, bbox_inches="tight")
plt.close()

print("Saved: shap_global_top10.png, shap_local_waterfall.png")

Saved: shap_global_top10.png, shap_local_waterfall.png
